# S.Ma.R.T. MODEL TRAINING

In [1]:
# Se define el estilo de visualización del notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import time
from datetime import timedelta
start_time = time.time()

In [3]:
import datetime as dt
import numpy    as np
import pandas   as pd 
import scipy

extrae_id = lambda x : (x.strip('ObjectId').strip('(")'))

def remove_outliers_6sigma(X):
    return X[abs(X - np.mean(X)) < 3 * np.std(X)]

In [4]:
ruta = "./dataset/all_data.csv"
df   = pd.read_csv(ruta, converters = {'_id':extrae_id , 'DrillID':extrae_id})

In [5]:
df.sort_values(['TagID', 'DrillID', 'TimeStamp'], ascending=[True, True, True], inplace=True)

In [6]:
if df.duplicated().sum() > 0:
    df = df.drop_duplicates()

In [7]:
df.drop(list(df.columns[df.apply(pd.Series.nunique)==1]), axis=1, inplace=True)

In [8]:
if df.isnull().sum().sum() > 0:
    df = df.fillna(0)

In [9]:
df['PositionX'] = pd.to_numeric(df['Position'].str.strip('[]').str.split(pat="," , expand = True)[0], downcast='float')

In [10]:
df.drop(['Position'], axis=1, inplace=True)

In [11]:
df['TimeStampDT'] = pd.to_datetime(df['TimeStamp'], unit='ms')

In [12]:
df  = df[['_id', 'TagID', 'DrillID', 'TimeStamp', 'TimeStampDT', 'PositionX']]

In [13]:
df["diff_posx_m"]   = df["PositionX"].diff().abs()
df["diff_time_sec"] = df["TimeStamp"].diff().abs() / 1000
df["LinearSpeed"]   = df["diff_posx_m"].div(df["diff_time_sec"])
df["Acceleration"]  = df["diff_posx_m"].div(df["diff_time_sec"]**2)

In [14]:
if df.isnull().sum().sum() > 0:
    df = df.fillna(0)

In [15]:
df = df.drop( (df[df['LinearSpeed']  > 1.56].index ) )
df = df.drop( (df[df['Acceleration'] > 0.03].index ) )

In [16]:
variables = ["diff_posx_m", "diff_time_sec", "LinearSpeed", "Acceleration"]

In [17]:
df_aux1     = pd.merge(df     , pd.DataFrame(remove_outliers_6sigma(df[variables[0]].values)), how = 'left', left_index = True, 
                       right_index = True).rename(columns={0: variables[0] + '_no_outlier'})
df_aux2     = pd.merge(df_aux1, pd.DataFrame(remove_outliers_6sigma(df[variables[1]].values)), how = 'left', left_index = True, 
                       right_index = True).rename(columns={0: variables[1] + '_no_outlier'})
df_aux3     = pd.merge(df_aux2, pd.DataFrame(remove_outliers_6sigma(df[variables[2]].values)), how = 'left', left_index = True, 
                       right_index = True).rename(columns={0: variables[2] + '_no_outlier'})
df_filtered = pd.merge(df_aux3, pd.DataFrame(remove_outliers_6sigma(df[variables[3]].values)), how = 'left', left_index = True, 
                       right_index = True).rename(columns={0: variables[3] + '_no_outlier'})

df_filtered.dropna(inplace=True)

In [18]:
df_filtered.drop(variables, axis=1, inplace=True)
df_filtered.rename(columns={"diff_posx_m_no_outlier"  : "diff_pos",
                            "diff_time_sec_no_outlier": "diff_time",
                            "LinearSpeed_no_outlier"  : "Speed", 
                            "Acceleration_no_outlier" : "Accel"}, inplace=True)

In [19]:
df  = df_filtered.copy()

In [20]:
df_agg = df.groupby(['TagID', 'DrillID']).agg({'TimeStamp'  : ['min','max'],
                                               'TimeStampDT': ['min','max'], 
                                               'PositionX'  : ['min','max', 'mean'], 
                                               'diff_pos'   : ['min','max', 'mean'],
                                               'diff_time'  : ['min','max', 'mean'],
                                               'Speed'      : ['min','max', 'mean'],
                                               'Accel'      : ['min','max', 'mean']}).reset_index()

In [21]:
df6 = df_agg.copy()

In [22]:
df_agg = df_agg.drop(['TagID', 'DrillID', 'TimeStamp', 'TimeStampDT'], axis = 1)

C:\Users\calfo\Anaconda3\envs\env_ml\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [23]:
df_agg['Outlier_PositionX'] = ( df_agg['PositionX']['max'] * df_agg['PositionX']['min'] ) >= 0

In [24]:
df_mask_X  = df_agg['Outlier_PositionX']==False
filtered_X = df_agg[df_mask_X]
df_agg     = filtered_X.drop('Outlier_PositionX', axis=1)

In [25]:
df_agg.columns = ['_'.join(col_).strip() for col_ in df_agg.columns.values]

In [26]:
df_ul = df_agg.copy()

In [27]:
df_ul['classifier_'] = np.log( (1 + df_ul['Speed_mean']) / ( 1 + (df_ul['Accel_mean'] * df_ul['diff_time_mean']) ) )

In [28]:
df_ul.drop(['diff_time_max', 'Speed_mean'], axis=1, inplace=True)

In [29]:
from sklearn.preprocessing import MinMaxScaler
min_max           = MinMaxScaler()
df_scaled         = pd.DataFrame(min_max.fit_transform(df_ul.astype("float64")))
df_scaled.columns = df_ul.columns

In [30]:
from sklearn.cluster import KMeans
number_of_clusters = 3
kmeans             = KMeans(n_clusters = number_of_clusters)
kmeans.fit(df_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [31]:
import joblib
joblib.dump(kmeans, 'smart_new_model.pkl', compress=9)

['smart_new_model.pkl']

In [32]:
elapsed_time_secs = time.time() - start_time
msg = "Execution took: %s secs (Wall clock time)" % timedelta(seconds=round(elapsed_time_secs))
print(msg)

Execution took: 0:05:19 secs (Wall clock time)


In [33]:
!jupyter nbconvert --to pdf smart_model_training.ipynb

[NbConvertApp] Converting notebook smart_model_training.ipynb to pdf
[NbConvertApp] Writing 48220 bytes to .\notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', '.\\notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', '.\\notebook']
[NbConvertApp] WARNING | b had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 39739 bytes to smart_model_training.pdf
